<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Chat with documents using Generative AI with Vantage</b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction:</b></p>
<p style = 'font-size:16px;font-family:Arial'>In the Question-Answering system using Generative AI demo, the combination of <b>RAG, Langchain, and LLM models</b> allows users to ask queries in layman's terms, retrieve relevant information from the Vantage tables, and generate accurate and concise answers based on the retrieved data. This integration of retrieval-based and generative-based approaches provides a powerful tool for extracting knowledge from structured sources and delivering user-friendly responses.</p>

<p style = 'font-size:16px;font-family:Arial'>In this demo we will build Generative Question-Answering using LangChain, a powerful library for working with LLMs like GPT-3.5, GPT-4, Bloom, etc. and JumpStart in ClearScape notebooks, a system is built where users can ask business questions in natural English and receive answers with data drawn from the relevant databases.</p>

<p style = 'font-size:16px;font-family:Arial'>The following diagram illustrates the architecture.</p>

<center><img src="images/header2.png" alt="architecture"  width=800 height=800/></center>

<br>
<p style = 'font-size:16px;font-family:Arial'>Before going any farther, let's get a better understanding of RAG, LangChain, and LLM.</p>

<ul style = 'font-size:16px;font-family:Arial'><li> <b>Retrieval-Augmented Generation (RAG):</b></li></ul>
<p style = 'font-size:16px;font-family:Arial'> &emsp;  &emsp;RAG is a framework that combines the strengths of retrieval-based and generative-based approaches in question-answering systems.It utilizes both a retrieval model and a generative model to generate high-quality answers to user queries. The retrieval model is responsible for retrieving relevant information from a knowledge source, such as a database or documents. The generative model then takes the retrieved information as input and generates concise and accurate answers in natural language.</p>

<ul style = 'font-size:16px;font-family:Arial'><li> <b>Langchain:</b></li></ul>
<p style = 'font-size:16px;font-family:Arial'> &emsp;  &emsp; LangChain is a framework that facilitates the integration and chaining of large language models with other tools and sources to build more sophisticated AI applications. LangChain does not serve its own LLMs; instead, it provides a standard way of communicating with a variety of LLMs, including those from OpenAI and HuggingFace. LangChain accelerates the development of AI applications with building blocks. We learn the leverage the following building blocks in this notebook:</p>
 
<ol style = 'font-size:16px;font-family:Arial'>
    <li> <b> LLMs</b> – LangChain's <code>llm</code> class is designed to provide a standard interface for all LLM it supports.   </li>
    <li> <b> PromptTemplate</b>  - LangChain’s <code>PromptTemplate</code> class are predefined structures for generating prompts for LLM’s. They can be reused across different LLM's.</li>
    <li> <b> Chains</b> – When we build complex AI applications, we may need to combine multiple calls to LLM’s and to other components  LangChain’s <code>chain</code> class allows us to link calls to LLM’s and components. The most common type of chaining in any LLM application is combining a prompt template with an LLM and optionally an output parser. </li>
</ol>

<ul style = 'font-size:16px;font-family:Arial'><li> <b>LLM Models (Large Language Models):</b></li></ul>
<p style = 'font-size:16px;font-family:Arial'> &emsp;  &emsp; LLM models refer to the large-scale language models that are trained on vast amounts of text data.
These models, such as GPT-3 (Generative Pre-trained Transformer 3),  GPT-3.5, GPT-4, HuggingFace BLOOM, LLaMA, Google's FLAN-T5, etc. are capable of generating human-like text responses. LLM models have been pre-trained on diverse sources of text data, enabling them to learn patterns, grammar, and context from a wide range of topics. They can be fine-tuned for specific tasks, such as question-answering, natural language understanding, and text generation.
LLM models have achieved impressive results in various natural language processing tasks and are widely used in AI applications for generating human-like text responses.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Configuring the environment</li>
    <li>Connect to Vantage</li>
    <li>Data Exploration</li>
    <li>LLM</li>
    <li>Run the query function</li>
    <li>Cleanup</li>
</ol>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>1. Configuring the environment</b>

In [ ]:
%%capture
# '%%capture' suppresses the display of installation steps of the following packages
!pip install -r requirements.txt --quiet

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial'><b>Note: </b><i>The above statements will install the required libraries to run this demo. Be sure to restart the kernel after executing the above lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
    </div>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>1.1 Import the required libraries</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import io
import os

import numpy as np
import pandas as pd

# teradata lib
from teradataml import *

# LLM
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import format_document
from langchain.schema.runnable import RunnableMap
import panel as pn
from langchain.llms import OpenAI

# Suppress warnings
warnings.filterwarnings('ignore')
display.max_rows = 5

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>2. Connect to Vantage and OpenAI</b>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>2.1 Get the OpenAI API key</b></p>

<p style = 'font-size:16px;font-family:Arial'>In order to utilize this demo, you will need an OpenAI API key. If you do not have one, please refer to the instructions provided in this guide to obtain your OpenAI API key: </p>

[Openai_setup_api_key_guide](..//Openai_setup_api_key/Openai_setup_api_key.md)

In [ ]:
# enter your openai api key
api_key = input(prompt = '\n Please Enter OpenAI API key: ')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>3. Data Exploration</b>

<p style = 'font-size:16px;font-family:Arial'>The Chat with documentation demo aims to demonstrate how users can interact with documents such as insurance policy wordings, invoices, and other similar documents through a conversational interface.</p>

<p style = 'font-size:16px;font-family:Arial'>The Traveller Easy Single Trip - International insurance policy is a comprehensive travel insurance plan that provides cover for a wide range of risks, including medical expenses, trip cancellation, loss of luggage, and personal accident. The policy is designed to be affordable and flexible, and it can be purchased online or over the phone.<p/>


<p style = 'font-size:16px;font-family:Arial'>The source data from <a href="https://axa-com-my.cdn.axa-contento-118412.eu/axa-com-my/3d2f84a5-42b9-459b-911a-710546df0633_Policy+wording+-+SmartTraveller+Easy+Single+Trip+-+International+%280820%29.pdf">AXA</a> is loaded in FAISS as Vector Database.</p>

<p style = 'font-size:16px;font-family:Arial'>Now, let's use <code>PyPDFLoader</code> library to read the pdf document and split it into pages.</p>


In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/SmartTraveller_International.pdf")
pages = loader.load_and_split()

<p style = 'font-size:16px;font-family:Arial'>There are 24 pages that describe the policy in detail.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>4. LLM </b>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>4.1 Define LLM model</b></p>  

<p style = 'font-size:16px;font-family:Arial'>In OpenAI's language models, the <b>temperature</b> parameter controls the randomness of the generated text. It affects the diversity and creativity of the model's responses. It is always a number between 0 and 1. A temperature of 0 means the responses will be very straightforward, almost deterministic (meaning you almost always get the same response to a given prompt). A temperature of 1 means the responses can vary wildly.</p>


<p style = 'font-size:16px;font-family:Arial'>A higher temperature value, such as 1.0, increases the randomness and diversity of the generated output. This can lead to more varied and surprising responses, but it may also result in less coherence and occasional nonsensical outputs. A higher temperature means that the model might select a word with slightly lower probability, leading to more variation, randomness and creativity. A very high temperature therefore increases the risk of <b>hallucination</b>, meaning that the model starts selecting words that will make no sense or be offtopic.</p>

<p style = 'font-size:16px;font-family:Arial'>On the other hand, a lower temperature value, such as 0.2 or below, reduces randomness and makes the model's output more focused and deterministic. The generated text is likely to be more conservative, sticking closely to patterns observed in the training data. A temperature of 0 means roughly that the model will always select the highest probability word.</p>

<p style = 'font-size:16px;font-family:Arial'>Choosing an appropriate temperature value depends on the desired output. Higher temperatures can be useful for creative tasks or brainstorming, while lower temperatures are preferred when you need more control over the output, such as when generating specific responses or following a particular style.</p>

In [ ]:
from langchain.llms import OpenAI

# OpenAI API
os.environ["OPENAI_API_KEY"] = api_key
    
# call open AI model - api
llm = OpenAI(temperature=0) 

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b> 4.4 Define the function to generate response to user query</b></p>

<p style = 'font-size:16px;font-family:Arial'>Next, we run LangChain’s <code>SQLDatabaseChain</code> to convert text to SQL and implicitly run the generated SQL against the database to retrieve the database results in a simple readable language. we are taking the following steps:</p>

<ol style = 'font-size:16px;font-family:Arial'> 
    <li>Let's begin by establishing a prompt template that guides the Language Model (LLM) to generate SQL statements in a dialect that adheres to correct syntax. Subsequently, we execute these generated statements against the corresponding database.</li>
    <li>Finally, We utilize the <code>SQLDatabaseChain</code> to execute the SQL query, passing it the LLM, database connection, and prompt as inputs. Then, we take the SQL results and pass them back to the LLM to generate a response for the user's query.</li>
</ol>

In [ ]:
vectorstore = FAISS.from_documents(pages, OpenAIEmbeddings())

vectorstore.save_local("./data/faiss_index")

In [ ]:
retriever =  vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [ ]:
from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [ ]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [ ]:
from typing import List, Tuple


def _format_chat_history(chat_history: List[Tuple]) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        human = "Human: " + dialogue_turn[0]
        ai = "Assistant: " + dialogue_turn[1]
        buffer += "\n" + "\n".join([human, ai])
    return buffer

In [ ]:
_inputs = RunnableMap(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: _format_chat_history(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
)
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI()

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>5. Chat with documents</b>

<p style = 'font-size:16px;font-family:Arial'>Run the run_query function that in turn calls the Langchain SQL Database chain to convert 'text to sql' and runs the query against the source database</p>

In [ ]:
import panel as pn

pn.extension(design="material")

chat_history = []
def get_window_chats(k=3):
    return chat_history[-k:]

def callback(contents, user, instance):
    result = conversational_qa_chain.invoke({"question": contents,"chat_history": get_window_chats()}).content
    chat_history.append((contents, result))    
    return result


pn.chat.ChatInterface(callback=callback, show_rerun = False, show_undo = False, show_clear = False, width=800, height=400)

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>6. Cleanup</b>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
remove_context()

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">Copyright © Teradata Corporation - 2023. All Rights Reserved.</footer>